### imports && loading data

In [1]:
import os, sys
from RAI.dataset import Data, Dataset
from RAI.AISystem import AISystem, Model, Task
from RAI.utils import df_to_RAI
from RAI.torch import TorchRaiDB
from RAI.redis import RaiRedis
import numpy as np
import pandas as pd
import torch  
from torch import nn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
 
use_dashboard  = True
np.random.seed(21)
from RAI.utils import Reweighing
 

 

data_path = "./data/adult/"

train_data = pd.read_csv(data_path+"train.csv", header=0,
                    skipinitialspace=True, na_values="?")
test_data = pd.read_csv(data_path+"test.csv", header=0,
                skipinitialspace=True, na_values="?")
all_data = pd.concat( [train_data, test_data],ignore_index=True)

mdl = RandomForestClassifier(n_estimators=5, min_samples_leaf=20, max_depth=2)
#convert aggregated data into RAI format
meta, X,y  = df_to_RAI(all_data, target_column = "income-per-year", normalize = None, max_categorical_threshold = 5)
xTrain, xTest, yTrain, yTest = train_test_split(X, y, random_state=1, stratify=y)

#%%  
# Create a model to make predictions
model = Model(agent=mdl, model_class="Random Forest Classifier")
task = Task(model=model, type='binary_classification', description="Detect Cancer in patients using skin measurements")
configuration = {"fairness": {"priv_group": {"race": {"privileged": 1, "unprivileged": 0}},
                                "protected_attributes": ["race"], "positive_label": 1},
                    "time_complexity": "polynomial"}

dataset = Dataset(  train_data = Data(xTrain , yTrain), 
                        test_data = Data(xTest , yTest)) 

### Initialize RAI

In [2]:
ai = AISystem("AdultDB_Test1", meta_database=meta, dataset=dataset, task=task )
ai.initialize(user_config=configuration)
if use_dashboard:
    r = RaiRedis( ai )
    r.reset_redis()

metric group : metadata was loaded
metric group : Tree Models was loaded
metric group : performance_cl was loaded
metric group : summary_stats was loaded
metric group : stat_moment_group was loaded
metric group : frequency_stats was loaded
metric group : correlation_stats_binary was loaded
metric group : basic_robustness was loaded
metric group : noise_robustness was loaded
metric group : adversarial_robustness was loaded
metric group : group_fairness was loaded
metric group : individual_fairness was loaded
metric group : dataset_fairness was loaded
metric group : prediction_fairness was loaded
metric group : sample_distortion_fairness was loaded
metric group : basic_explainablity was loaded


### Model Fitting

In [3]:
#%%

mdl.fit(xTrain,yTrain)

ai.compute( mdl.predict(xTest), data_type="test", tag="Random Forest")
r.add_measurement()
        
 

### Reweighing

In [4]:
Reweighing()
mdl.fit(xTrain,yTrain)

ai.compute( mdl.predict(xTest), data_type="test", tag="Random Forest with Reweighting")
r.add_measurement() 
